In [1]:
using Pkg
Pkg.activate("libs/")

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


In [2]:
using JLD2
using DICOM
using Dates
using XLSX
using Plots
using Images
using ProgressBars

In [7]:
BAC_root_dir = "/media/molloi-lab/2TB/BAC_Full_Data"
excel_path = "/home/molloi-lab/Desktop/Project BAC/BAC project/Ca mass calculation Final.xlsx"
V_P_dict = Dict("L CC" => 6, "L MLO" => 2, "R CC" => 14, "R MLO" => 10)

image_dirs = ["Patientstudy_BAC_2013a/data_from_KP", 
"Patientstudy_BAC_2013b/data_from_KP", 
"Patientstudy_BAC_2014/data_from_KP", 
"Patientstudy_BAC_2015a/data_from_KP", 
"Patientstudy_BAC_2015b/data_from_KP", 
"Patientstudy_BAC_2016/Data from KP"]

save_dir = "/media/molloi-lab/2TB/Clean_Dataset_full";

In [4]:
@load "result_from_step1.jld" output_csv

1-element Vector{Symbol}:
 :output_csv

In [24]:
idxes = [2, 3, 6, 7, 10, 11, 14, 15]
for (i, data_) in enumerate(output_csv)
    for j in idxes
        for (k, str) in enumerate(output_csv[i][j])
            output_csv[i][j][k] = joinpath(BAC_root_dir, str[52 : end])
        end
    end
end

# Helper functions
output csv format:

1: SID

2: path to L MLO image  
3: path to L MLO roi  
4: num_patch_with_BAC  
5: num_patch_without_BAC

6: path to L CC image  
7: path to L CC roi  
8: num_patch_with_BAC  
9: num_patch_without_BAC

10: path to R MLO image  
11: path to R MLO roi  
12: num_patch_with_BAC  
13: num_patch_without_BAC

14: path to R CC image  
15: path to R CC roi  
16: num_patch_with_BAC  
17: num_patch_without_BAC

18: total_num_patch_with_BAC  
19: total_num_patch_without_BAC

In [27]:
function copy_dicom_file(old_path::String, new_path::String)
    # Check if the old DICOM file exists
    if !isfile(old_path)
        println("Error: The old DICOM file does not exist.")
        return
    end
    
    # Check if the new path directory exists, if not create it
    new_dir = dirname(new_path)
    if !isdir(new_dir)
        # println("Creating new directory: ", new_dir)
        mkpath(new_dir)
    end
    
    # Copy the DICOM file
    if isfile(new_path)
        return
    end
    try
        cp(old_path, new_path)
        # println("Successfully copied DICOM file to new path.")
    catch e
        println("An error occurred while copying the DICOM file: ", e)
    end
end

copy_dicom_file (generic function with 1 method)

# 4. Create PNG file for masks; Copy to clean folder

In [28]:
# runtime ~= 170s
strs = ["L_MLO", "L_CC", "R_MLO", "R_CC"]
Threads.@threads for i in ProgressBar(1 : size(output_csv)[1], unit="Patient")
    SID = output_csv[i][1]
    # Create output folder
    curr_save_dir = joinpath(save_dir, SID)
    # isdir(curr_save_dir) || mkdir(curr_save_dir)
    # Process each image
    for j = 1:4
        View_position = strs[j]
        img_path_idx = (j-1)*4+2
        idx_num_patch_with_BAC, idx_num_patch_without_BAC = img_path_idx+2, img_path_idx+3
        img_paths = output_csv[i][img_path_idx]
        roi_paths = output_csv[i][img_path_idx + 1]
        num_pairs = size(img_paths)[1]
        for k = 1:num_pairs
            curr_img_path = img_paths[k]
            curr_roi_path = roi_paths[k]
            # image
            img = dcm_parse(curr_img_path)[(0x7fe0, 0x0010)]
            s = size(img)
            # png mask
			mask = zeros(Gray{N0f8}, s)
            # set output path
            # file_name = splitext(rsplit(curr_img_path, '/'; limit = 2)[2])[1]
            file_name = View_position
            if k != 1
                file_name *= "_$k"
            end
            file_name *= ".$(s[1])_$(s[2])"
            save_img_path = joinpath(curr_save_dir, file_name*".dcm")
            if curr_roi_path != "zero BAC"
                save_roi_path = joinpath(curr_save_dir, file_name*".roi")
                copy_dicom_file(curr_roi_path, save_roi_path)
            end
            save_mask_path = joinpath(curr_save_dir, file_name*".png")
            # save
            save(save_mask_path, mask)
            # copy_dicom_file(curr_img_path, save_img_path)
        end
    end
end

Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not

0.0%┣                      ┫ 1Patient/5.1kPatient [00:00<Inf:Inf, InfGs/Patient]



Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does no

1.3%┣▎                       ┫ 65Patient/5.1kPatient [00:01<00:44, 116Patient/s]
1.4%┣▍                        ┫ 72Patient/5.1kPatient [00:01<00:54, 94Patient/s]
1.5%┣▍                        ┫ 75Patient/5.1kPatient [00:01<01:01, 82Patient/s]
1.6%┣▍                        ┫ 81Patient/5.1kPatient [00:01<01:00, 84Patient/s]
1.6%┣▍                        ┫ 83Patient/5.1kPatient [00:01<01:03, 79Patient/s]
2.0%┣▌                       ┫ 103Patient/5.1kPatient [00:01<00:53, 94Patient/s]
2.1%┣▌                       ┫ 107Patient/5.1kPatient [00:01<00:54, 92Patient/s]
2.1%┣▌                       ┫ 109Patient/5.1kPatient [00:01<00:55, 90Patient/s]



Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does no

...

52.6%┣███████████▋          ┫ 2.7kPatient/5.1kPatient [00:41<00:37, 65Patient/s]
52.6%┣███████████▋          ┫ 2.7kPatient/5.1kPatient [00:41<00:37, 65Patient/s]
52.9%┣███████████▋          ┫ 2.7kPatient/5.1kPatient [00:41<00:37, 65Patient/s]
52.9%┣███████████▋          ┫ 2.7kPatient/5.1kPatient [00:42<00:37, 65Patient/s]


Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not exist.
Error: The old DICOM file does not

53.0%┣███████████▋          ┫ 2.7kPatient/5.1kPatient [00:42<00:37, 65Patient/s]
53.1%┣███████████▊          ┫ 2.7kPatient/5.1kPatient [00:42<00:37, 65Patient/s]
53.4%┣███████████▊          ┫ 2.7kPatient/5.1kPatient [00:42<00:37, 65Patient/s]
53.5%┣███████████▉          ┫ 2.7kPatient/5.1kPatient [00:42<00:36, 65Patient/s]


Excessive output truncated after 524654 bytes.

53.6%┣███████████▉          ┫ 2.7kPatient/5.1kPatient [00:42<00:36, 65Patient/s]
53.7%┣███████████▉          ┫ 2.7kPatient/5.1kPatient [00:42<00:36, 65Patient/s]
53.9%┣███████████▉          ┫ 2.8kPatient/5.1kPatient [00:42<00:36, 65Patient/s]
54.0%┣███████████▉          ┫ 2.8kPatient/5.1kPatient [00:42<00:36, 65Patient/s]


In [8]:
csv_output = nothing
GC.gc(true)